### For binary classifcation

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [28]:
# Importing the dataset
churn_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [29]:
churn_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [30]:
churn_data.shape

(7043, 21)

In [31]:
churn_data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [32]:
churn_data.drop('customerID', axis=1, inplace=True)

In [33]:
# Handle missing values if any
churn_data['TotalCharges'] = pd.to_numeric(churn_data['TotalCharges'], errors='coerce')
churn_data.fillna(churn_data['TotalCharges'].mean(), inplace=True)

In [34]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [35]:
# Encode categorical variables
for column in churn_data.select_dtypes(include=['object']).columns:
    if column != 'Churn':
        churn_data[column] = LabelEncoder().fit_transform(churn_data[column])


In [36]:

# Encode the target variable
churn_data['Churn'] = churn_data['Churn'].map({'No': 0, 'Yes': 1})


In [37]:
churn_data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1


In [38]:
data=churn_data.copy()

In [39]:
# Split into features and target
X = data.drop('Churn', axis=1).values
y = data['Churn'].values
X, y

(array([[0.0000e+00, 0.0000e+00, 1.0000e+00, ..., 2.0000e+00, 2.9850e+01,
         2.9850e+01],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 3.0000e+00, 5.6950e+01,
         1.8895e+03],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 3.0000e+00, 5.3850e+01,
         1.0815e+02],
        ...,
        [0.0000e+00, 0.0000e+00, 1.0000e+00, ..., 2.0000e+00, 2.9600e+01,
         3.4645e+02],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, ..., 3.0000e+00, 7.4400e+01,
         3.0660e+02],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 1.0565e+02,
         6.8445e+03]]),
 array([0, 0, 1, ..., 0, 1, 0], dtype=int64))

In [40]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Build the ANN Model:

In [42]:
# Initialize the model
model = Sequential()

# Add input and first hidden layer with dropout
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))

# Add second hidden layer with dropout
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))

# Add output layer
model.add(Dense(1, activation='sigmoid'))

c:\Users\s21\Desktop\my data\DeepLearning Frameworks\Tensorflow_Keras_Pytorch\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 32)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,185 (4.63 KB)

 Trainable params: 1,185 (4.63 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)


Epoch 1/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5564 - loss: 0.7705 - val_accuracy: 0.7801 - val_loss: 0.4667
Epoch 2/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - accuracy: 0.7165 - loss: 0.5544 - val_accuracy: 0.7890 - val_loss: 0.4423
Epoch 3/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - accuracy: 0.7427 - loss: 0.5092 - val_accuracy: 0.7926 - val_loss: 0.4351
Epoch 4/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.7490 - loss: 0.4962 - val_accuracy: 0.7908 - val_loss: 0.4322
Epoch 5/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.7584 - loss: 0.4867 - val_accuracy: 0.7872 - val_loss: 0.4273
Epoch 6/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - accuracy: 0.7527 - loss: 0.4826 - val_accuracy: 0.7943 - val_loss: 0.4252
Epoch 7/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.7636 - loss: 0.4846 - val_accuracy: 0.8014 - val_loss: 0.4224
Epoch 8/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - accuracy: 0.7608 - loss: 0.4567 - va

#### Model prediction on test data

In [70]:
y_pred=model.predict(X_test)
y_pred

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step


array([[0.46835083],
       [0.13243675],
       [0.00245007],
       ...,
       [0.04691143],
       [0.02158533],
       [0.3822583 ]], dtype=float32)

In [74]:
for j, i in enumerate(y_pred):
    print(i[0])
    if j==5:
        break

0.46835083
0.13243675
0.0024500743
0.5043582
0.0050163595
0.26521495


#### Evaluation model

In [45]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - accuracy: 0.7995 - loss: 0.4084
Loss: 0.4032202959060669, Accuracy: 0.8105039000511169


### Predict on Unseen Data:

If you have new data to predict on, ensure it undergoes the same preprocessing steps.

In [ ]:
# Load unseen data
unseen_data = pd.read_csv('path_to_unseen_data.csv')

# Drop the 'customerID' column
unseen_data.drop('customerID', axis=1, inplace=True)

# Handle missing values and encode categorical variables
unseen_data['TotalCharges'] = pd.to_numeric(unseen_data['TotalCharges'], errors='coerce')
unseen_data.fillna(unseen_data['TotalCharges'].mean(), inplace=True)

for column in unseen_data.select_dtypes(include=['object']).columns:
    unseen_data[column] = LabelEncoder().fit_transform(unseen_data[column])

# Feature scaling
unseen_data_scaled = scaler.transform(unseen_data.values)

# Predict
predictions = model.predict(unseen_data_scaled)
predictions = (predictions > 0.5).astype(int)  # Convert probabilities to binary output

# Save predictions to CSV
predictions_df = pd.DataFrame(predictions, columns=['Predicted'])
predictions_df.to_csv('predictions.csv', index=False)
